# Set Up The Environment

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import re
import os
import time
from tqdm.auto import tqdm
from ckiptagger import WS, POS, NER, construct_dictionary # tokenization

# Setup work directory
os.chdir('/Users/deankuo/Desktop/python/dissertation/Taiwan-Manifesto-Analysis-A-ML-Approach')
os.listdir()

# Import utils.py
from Utils.utils_token import (
    load_text,
    split_content,
    tokenization,
    postprocess_dataframe,
    
)
matplotlib.rcParams['font.family'] = 'Times New Roman'

# CKIP module
CKIP_PATH = "../CKIP_TAGGER"
ws = WS(CKIP_PATH) # 斷詞
pos = POS(CKIP_PATH) # 詞性標註
ner = NER(CKIP_PATH) # 命名實體識別

2024-04-25 11:01:50.698634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/deankuo/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2024-04-25 11:02:00.986005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with t

KeyboardInterrupt: 

In [ ]:
# 加入自定義字典
word_to_weight = {'823': 1, 'ECFA': 1, '2300': 1, '台26線': 1, '台74線': 1, '12年國教': 1, 'BOT': 1, '88快速道路': 1, '台27線': 1, '台61線': 1, '十二年國教': 1, '國道10號': 1,
                  '台88號': 1, 'M型': 1, '205兵工廠': 1, '北二高': 1, '台65線': 1, 'CEPA': 1, 'FTA': 1, '科學園區': 1, '228': 1, 'MIT': 1, '202兵工廠': 1, '86快速道路': 1, '國道8號': 1,
                  '台64': 1, '台66': 1, 'iBike': 1, 'MRT': 1, 'TPP': 1, 'TIFA': 1, 'TPP':1, '台22': 1, '台29': 1, '國10': 1, '國1': 1, '318': 1, 'NCC':1, 'PM2.5': 1, 'YouBike': 1, 
                  '台68': 1, '快速道路': 1, 'NGO': 1, 'NPO': 1, 'U-Bike': 1, 'LGBTQ': 1, '三七五減租': 1, '小三通': 1, '大三通': 1, '基礎建設': 1, '戒急用忍': 1, '社會役': 1, '非核家園': 1,
                  '教育券': 1, '九二共識': 1}
dictionary = construct_dictionary(word_to_weight)

# 停用詞
with open("./Data/stopwords_zh-tw.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[1:]

In [2]:
years = [1992, 1995, 1998, 2001, 2004, 2008, 2012, 2016, 2020, 2024]
dfs = {}
num = []
for year in years:
    dfs[year] = pd.read_excel(f"./Data/merge_data/{year}.xlsx")
    print(dfs[year].shape)
    num.append(dfs[year].shape[0])

(330, 17)
(314, 17)
(383, 17)
(434, 17)
(368, 17)
(283, 17)
(267, 17)
(354, 17)
(410, 17)
(309, 17)


In [3]:
for k, v in dfs.items():
    dfs[k] = split_content(dfs[k])

In [4]:
sum = 0
n = 0
for k, v in dfs.items():
    t = v.shape[0]
    sum += t
    print(f"{k} 年的選舉共有 {t} 筆資料")
    print(f"{k} 年選舉的候選人平均提出 {round(t / num[n], 2)} 筆政見\n")
    n += 1
print(f"一共有 {sum} 筆資料")
    

1992 年的選舉共有 3970 筆資料
1992 年選舉的候選人平均提出 12.03 筆政見

1995 年的選舉共有 3580 筆資料
1995 年選舉的候選人平均提出 11.4 筆政見

1998 年的選舉共有 4332 筆資料
1998 年選舉的候選人平均提出 11.31 筆政見

2001 年的選舉共有 4647 筆資料
2001 年選舉的候選人平均提出 10.71 筆政見

2004 年的選舉共有 3926 筆資料
2004 年選舉的候選人平均提出 10.67 筆政見

2008 年的選舉共有 3724 筆資料
2008 年選舉的候選人平均提出 13.16 筆政見

2012 年的選舉共有 2917 筆資料
2012 年選舉的候選人平均提出 10.93 筆政見

2016 年的選舉共有 3918 筆資料
2016 年選舉的候選人平均提出 11.07 筆政見

2020 年的選舉共有 4797 筆資料
2020 年選舉的候選人平均提出 11.7 筆政見

2024 年的選舉共有 3744 筆資料
2024 年選舉的候選人平均提出 12.12 筆政見

一共有 39555 筆資料


# Tokenization

In [5]:
# Test the text selection function
text = dfs[2024].loc[250, 'CONTENT']

sentence_, pos_, ner_, token_ = load_text(text, test=1)
print()
print('=====')
print("原文：")
print(text)
print("斷詞後：")
print(sentence_)
print("斷詞後+詞性標注：")
print(pos_)
print("透過詞性篩選的斷詞: ")
print(token_)
print("命名實體辨識: ")
print(ner_)
print('=====')


=====
原文：
５、青年＆就業︰規劃吸引年輕人返鄉就業，本區應制定在地廠商聘用本區或外地的年輕人之相關優惠的補助條件和方式，讓年輕人願意留在本區務農、就業、甚至創業。
斷詞後：
青年 就業 規劃 吸引 年輕人 返鄉 就業 制定 在地 廠商 聘用 外地 年輕人 相關 優惠 補助 條件 方式 年輕人 願意 留在 務農 就業 甚至 創業
斷詞後+詞性標注：
['青年(Na)', '就業(VA)', '規劃(VC)', '吸引(VJ)', '年輕人(Na)', '返鄉(VA)', '就業(VA)', '制定(VC)', '在地(A)', '廠商(Na)', '聘用(VC)', '外地(Nc)', '年輕人(Na)', '相關(VH)', '優惠(VJ)', '補助(Nv)', '條件(Na)', '方式(Na)', '年輕人(Na)', '願意(VK)', '留在(VCL)', '務農(VA)', '就業(VA)', '創業(VA)']
透過詞性篩選的斷詞: 
青年 就業 規劃 吸引 年輕人 返鄉 就業 制定 在地 廠商 聘用 外地 年輕人 相關 優惠 補助 條件 方式 年輕人 願意 留在 務農 就業 創業
命名實體辨識: 
[{(0, 1, 'CARDINAL', '５')}]
=====


## Iterate Each Year Data

In [6]:
for key, value in dfs.items():
    dfs[key] = tokenization(key, value)

Tokenizing 1992 election statements:   0%|          | 0/3970 [00:00<?, ?it/s]

1992年選舉公報的斷詞運算時間為: 8.94 分


Tokenizing 1995 election statements:   0%|          | 0/3580 [00:00<?, ?it/s]

1995年選舉公報的斷詞運算時間為: 6.78 分


Tokenizing 1998 election statements:   0%|          | 0/4332 [00:00<?, ?it/s]

...

2016年選舉公報的斷詞運算時間為: 7.92 分


Tokenizing 2020 election statements:   0%|          | 0/4797 [00:00<?, ?it/s]

2020年選舉公報的斷詞運算時間為: 8.59 分


Tokenizing 2024 election statements:   0%|          | 0/3744 [00:00<?, ?it/s]

2024年選舉公報的斷詞運算時間為: 7.1 分


## Post Processing

In [3]:
years = [1992, 1995, 1998, 2001, 2004, 2008, 2012, 2016, 2020, 2024]
dfs = {}
for year in years:
    dfs[year] = pd.read_csv(f"./Data/Dataset/{year}.csv")

In [7]:
for k, v in dfs.items():
    dfs[k] = postprocess_dataframe(dfs[k])

In [8]:
sum = 0
for k, v in dfs.items():
    sum += dfs[k].shape[0]

print(sum)

39535


In [9]:
for k, v in dfs.items():
    dfs[k].to_csv(f'./Data/Dataset/{k}.csv', encoding='utf-8-sig', index=False)

In [10]:
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df.to_csv('./Data/Manifesto_Dataset.csv', encoding='utf-8-sig', index=False)